# HKU QIDS 2023 Quantitative Investment Competition: Preprocess

## Init Config

In [11]:
import pandas as pd
import copy
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

In [12]:
seed = 257248
stock_num = 54
train_day_num_total = 1000
train_day_num = 1000 - 2
test_day_num = 700
timeslot_num = 50

In [13]:
calc_mean = lambda df: df.mean(axis=0)
calc_max = lambda df: df.max(axis=0)
calc_min = lambda df: df.min(axis=0)
calc_std = lambda df: df.std(axis=0)
calc_add = lambda df1, df2: df1 + df2
calc_diff = lambda df1, df2: df1 - df2
calc_prod = lambda df1, df2: df1 * df2
calc_div = lambda df1, df2: df1 / df2

In [14]:
def preprocess(fun, mar, ret=None):
    fun["stock_id"] = fun["date_time"].apply(lambda x: x.split("d")[0][1:]).astype("int")
    fun["day"] = fun["date_time"].apply(lambda x: x.split("d")[1][:]).astype("int")
    fun = fun.sort_values(by=["stock_id", "day"])
    na_fun = fun.loc[fun["day"].isin([999, 1000])]
    fun = fun.drop(na_fun.index, axis=0).reset_index(drop=True)
    na_fun = na_fun.reset_index(drop=True)

    mar["stock_id"] = mar["date_time"].apply(lambda x: x.split("d")[0][1:]).astype("int")
    mar["day"] = mar["date_time"].apply(lambda x: x.split("d")[1].split("p")[0]).astype("int")
    mar["time"] = mar["date_time"].apply(lambda x: x.split("p")[1]).astype("int")
    mar = mar.sort_values(by=["stock_id", "day", "time"]).reset_index(drop=True)
    na_mar = mar.loc[mar["day"].isin([999, 1000])]
    mar = mar.drop(na_mar.index, axis=0).reset_index(drop=True)
    na_mar = na_mar.reset_index(drop=True)

    combined = copy.deepcopy(fun)
    if ret is not None:
        ret["stock_id"] = ret["date_time"].apply(lambda x: x.split("d")[0][1:]).astype("int")
        ret["day"] = ret["date_time"].apply(lambda x: x.split("d")[1][:]).astype("int")
        ret = ret.sort_values(by=["stock_id", "day"]).reset_index(drop=True)
        combined["return"] = ret["return"]
        day_num = train_day_num
    else:
        day_num = test_day_num

    mar_summary = []
    start = 0
    for stock in range(stock_num):
        end = start + day_num * timeslot_num
        stock_info = mar.iloc[start:end, :]
        day_start = 0
        for day in range(day_num):
            day_end = day_start + timeslot_num
            stock_info_per_day = stock_info.iloc[day_start:day_end, :]
            mar_summary.append([
                calc_mean(stock_info_per_day["open"]),
                calc_mean(stock_info_per_day["close"]),
                calc_mean(stock_info_per_day["high"]),
                calc_mean(stock_info_per_day["low"]),
                calc_mean(stock_info_per_day["volume"]),
                calc_mean(stock_info_per_day["money"]),
                calc_max(stock_info_per_day["high"]),
                calc_max(stock_info_per_day["volume"]),
                calc_max(stock_info_per_day["money"]),
                calc_min(stock_info_per_day["low"]),
                calc_max(calc_div(calc_diff(stock_info_per_day["close"], stock_info_per_day["open"]), stock_info_per_day["open"])),
                calc_max(calc_div(calc_diff(stock_info_per_day["high"], stock_info_per_day["low"]), stock_info_per_day["open"])),
            ])
            day_start = day_end
        start = end
    cols = [
        "open_mean",
        "close_mean",
        "high_mean",
        "low_mean",
        "volume_mean",
        "money_mean",
        "high_max",
        "volume_max",
        "money_max",
        "low_min",
        "price_diff",
        "price_diff_max",
    ]
    mar_summary = pd.DataFrame(mar_summary, columns=cols)
    combined = pd.concat([combined, mar_summary], axis=1)

    return [combined, fun, mar, na_fun, na_mar, ret] if ret is not None else [combined, fun, mar, na_fun, na_mar]

In [15]:
def reorder(df):
    df_cols = df.columns
    if 'return' not in df_cols:
        df_cols_prior = ['date_time', 'stock_id', 'day']
    else:
        df_cols_prior = ['date_time', 'stock_id', 'day', 'return']
    for col in df_cols:
        if col not in df_cols_prior:
            df_cols_prior.append(col)
    if 'return' in df_cols_prior:
        df_cols_prior.remove('return')
        df_cols_prior.append('return')
    return df[df_cols_prior]

In [16]:
def train_valid_split(data_set, valid_ratio=0.2, seed=257248):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set, valid_set = train_test_split(data_set, test_size=valid_set_size, random_state=seed)
    return train_set, valid_set

In [17]:
def write(df, path):
    df.to_csv(path, index=False)

## Load Data

In [18]:
train_fun = pd.read_csv("../data/first_round_train_fundamental_data.csv")
train_mar = pd.read_csv("../data/first_round_train_market_data.csv")
train_ret = pd.read_csv("../data/first_round_train_return_data.csv")

In [19]:
test_fun = pd.read_csv("./qids_package/first_round_test_fundamental_data.csv")
test_mar = pd.read_csv("./qids_package/first_round_test_market_data.csv")

## Rearrange data by different stocks

In [20]:
train_combined, train_fun, train_mar, train_na_fun, train_na_mar, train_ret = preprocess(train_fun, train_mar, train_ret)

KeyError: 'date_time'

In [21]:
test_combined, test_fun, test_mar, test_na_fun, test_na_mar = preprocess(test_fun, test_mar)

KeyError: 'date_time'

## Reorder columns

In [97]:
train = reorder(train_combined)
test = reorder(test_combined)

## Split dataset

In [98]:
train, valid = train_valid_split(train)
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

## Write Files

In [100]:
write_path = "../data/"

train_path = write_path + "train.csv"
write(train, train_path)

valid_path = write_path + "valid.csv"
write(valid, valid_path)

test_path = write_path + "test.csv"
write(test, test_path)